In [1]:
import pandas as pd
df=pd.read_csv('Churn_Modelling.csv')
df

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [2]:
cols = ['RowNumber','CustomerId','Surname']
df.drop(cols,inplace=True,axis=1)

In [3]:
from sklearn.preprocessing import OrdinalEncoder

object_cols = ["Geography", "Gender"]

# Apply ordinal encoder to each column with categorical data
ordinal_encoder = OrdinalEncoder()
df[object_cols] = ordinal_encoder.fit_transform(df[object_cols])

In [4]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]
X

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,0.0,0.0,42,2,0.00,1,1,1,101348.88
1,608,2.0,0.0,41,1,83807.86,1,0,1,112542.58
2,502,0.0,0.0,42,8,159660.80,3,1,0,113931.57
3,699,0.0,0.0,39,1,0.00,2,0,0,93826.63
4,850,2.0,0.0,43,2,125510.82,1,1,1,79084.10
...,...,...,...,...,...,...,...,...,...,...
9995,771,0.0,1.0,39,5,0.00,2,1,0,96270.64
9996,516,0.0,1.0,35,10,57369.61,1,1,1,101699.77
9997,709,0.0,0.0,36,7,0.00,1,0,1,42085.58
9998,772,1.0,1.0,42,3,75075.31,2,1,0,92888.52


In [23]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

def perform_kfold_cross_validation(X, y, k):
    # Initialize lists to store the predictions, probabilities, and accuracy for each fold
    y_preds_list = []
    y_probs_list = []
    accuracies = []

    # Create k-fold cross-validator
    kf = KFold(n_splits=k, shuffle=True, random_state=42)

    # Split the data into k folds and perform cross-validation
    for train_idx, test_idx in kf.split(X):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # Create and train the model (you can replace LogisticRegression with your desired model)
        model = LogisticRegression()
        model.fit(X_train, y_train)

        # Make predictions using the model
        y_pred = model.predict(X_test)
        y_prob = model.predict_proba(X_test)[:, 1]  # Probabilities for class 1 (second value)

        # Calculate the accuracy for this fold
        accuracy = accuracy_score(y_test, y_pred)

        # Append predictions, probabilities, and accuracy to the respective lists
        y_preds_list.append(y_pred)
        y_probs_list.append(y_prob)
        accuracies.append(accuracy)

    return y_preds_list, y_probs_list, accuracies

# Example usage:
# Replace X and y with your data, and set k to the desired number of folds
# X is the input features and y is the target variable
X, y = np.random.rand(10000, 5), np.random.randint(2, size=10000)  # Example data (replace this with your data)
k = 5  # Number of folds for cross-validation
y_preds_list, y_probs_list, accuracies = perform_kfold_cross_validation(X, y, k)

# Combine the predictions and probabilities from all folds into a single array
y_pred_full = np.concatenate(y_preds_list)
y_prob_full = np.concatenate(y_probs_list)

# Create the new DataFrame
df_results = pd.DataFrame({'y_true': y, 'y_pred': y_pred_full, 'probability': y_prob_full})

print(df_results)


      y_true  y_pred  probability
0          0       0     0.495253
1          0       0     0.486748
2          0       0     0.493163
3          0       0     0.488134
4          0       0     0.493493
...      ...     ...          ...
9995       1       1     0.506514
9996       0       1     0.508912
9997       0       0     0.492851
9998       1       0     0.493952
9999       1       0     0.489380

[10000 rows x 3 columns]


In [24]:
accuracies

[0.4795, 0.5075, 0.4895, 0.484, 0.511]